In [ ]:
import pandas as pd

fake_news = pd.read_csv("Fake.csv")
true_news = pd.read_csv("True.csv")

In [ ]:
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


**Le fake news sono più frequenti in una determinata categoria?**

I dataset sono all'incirca della stessa lunghezza

In [ ]:
true_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [ ]:
fake_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


**Le fake news sono più frequenti in una determinata categoria?**

I temi come news e politics sono i piu ricorrenti per le fake news

In [ ]:
fake_news['subject'].value_counts()

,count
subject,
News,9050
politics,6841
left-news,4459
Government News,1570
US_News,783
Middle-east,778


In [ ]:
# @title Data Cleaner Function
import time
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
import string
from tqdm import tqdm
from nltk.corpus import stopwords
import spacy
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
# Creazione di stopwords e punteggiatura
english_stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# Caricamento del modello spaCy
nlp = spacy.load("en_core_web_sm")

# Inizializzazione del Lemmatizer di NLTK
lemmatizer = WordNetLemmatizer()

def data_cleaner(dataset):
    start_time_func = time.time()
    dataset_to_return = []

    for sentence in tqdm(dataset, desc="Cleaning data"):
        sentence = sentence.lower()
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))

        # Rimuove numeri
        sentence = re.sub('\d+', '', sentence)

        # Tokenizzazione e rimozione delle stopwords
        tokens = [word for word in nltk.word_tokenize(sentence) if word not in english_stopwords]

        # Lemmatizzazione
        processed_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])

        dataset_to_return.append(processed_text)

    return dataset_to_return


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
doc_text_false = data_cleaner(fake_news['text'])
doc_title_fake = data_cleaner(fake_news['title'])
doc_title_true = data_cleaner(true_news['title'])

Cleaning data: 100%|██████████| 21417/21417 [00:04<00:00, 5312.32it/s]


In [ ]:
doc_text_true = data_cleaner(true_news['text'])

Cleaning data: 100%|██████████| 21417/21417 [01:03<00:00, 336.23it/s]


# I titoli delle fake news presentano pattern ricorrenti?

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize the TF-IDF Vectorizer for trigrams (3-grams)
tfidf_vectorizer_ngram = TfidfVectorizer(ngram_range=(3, 3), stop_words="english")

# Fit and transform the dataset text
words_array_ngram = tfidf_vectorizer_ngram.fit_transform(doc_title_fake)

# Convert sparse matrix to dense format and create a DataFrame
words_dataframe_ngram = pd.DataFrame(words_array_ngram.toarray(), columns=tfidf_vectorizer_ngram.get_feature_names_out())

# Sum the scores of each trigram and sort by frequency
word_scores_ngram = words_dataframe_ngram.sum(axis=0).sort_values(ascending=False)

# Display the top 20 most frequent trigrams
print("Trigrammi più frequenti nelle fake news")
print(word_scores_ngram.head(20))


Trigrammi più frequenti nelle fake news
boiler room ep             46.122700
black life matter          30.394328
supreme court justice      13.194800
donald trump video         12.883113
new york time              12.362031
fox news host              12.141991
trump campaign manager     11.562907
donald trump jr            10.744143
patrick henningsen live     8.862422
watch donald trump          8.791874
episode sunday wire         8.437085
trump muslim ban            8.185802
president trump video       7.457900
ep patrick henningsen       7.376244
bernie sander supporter     7.045184
watch trump supporter       7.011945
trump supporter video       6.859956
make america great          6.822795
watch kellyanne conway      6.627192
watch fox news              6.608851
dtype: float64


i pattern più frequenti (analisi dei trigrammi) sono le combinazioni di parole:

- boiler room ep
- black life matter
- supreme court justice
- donald trump video

# Addestramento del modello

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

doc_fake = doc_text_false + doc_title_fake # unione dei titoli con il testo dei documenti fake
doc_true = doc_text_true + doc_title_true # unione dei titoli con il testo dei documenti true

dataset_text_fake = pd.DataFrame(doc_fake, columns=['text'])
dataset_text_fake['label'] = 0 # label che simboleggia i documenti fake

dataset_text_true = pd.DataFrame(doc_true, columns=['text'])
dataset_text_true['label'] = 1 # label che simboleggia i documenti real

data_text = pd.concat([dataset_text_fake, dataset_text_true], ignore_index=True) # unione dei due dataframe

X_test, X_train, y_test, y_train = train_test_split(data_text['text'], data_text['label'], test_size=0.2, random_state=42, shuffle=True)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Creazione di una pipeline che prima effettua la vettorizzazione tramite tfidf e poi applica la MultinomialNB
model = make_pipeline(TfidfVectorizer(stop_words="english"), MultinomialNB())

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calcolo metriche di valutazione
print("🔹 **Valutazione del modello**")
print(classification_report(y_test, y_pred)) # calcolo delle performance

🔹 **Valutazione del modello**
              precision    recall  f1-score   support

           0       0.92      0.94      0.93     37467
           1       0.93      0.90      0.92     34369

    accuracy                           0.92     71836
   macro avg       0.92      0.92      0.92     71836
weighted avg       0.92      0.92      0.92     71836



Il modello presenta ottime prestazioni, una accuratezza e f1 score del 92%, sottolineando la robustezza del modello

In [ ]:
# @title Esportazione in formato pikle

import pickle

with open("fake_news_model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)